In [15]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

In [11]:
train= pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [4]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
# target columns
labels = [col for col in train.columns if col not in ['id','comment_text']]

In [14]:
#append text comment from train and test
text = pd.concat([train['comment_text'],test['comment_text']], axis=0)

# word vectorize text comment by TFIDF
vectorizer_word = TfidfVectorizer(sublinear_tf=True, 
                             strip_accents='unicode',
                             analyzer='word',
                             ngram_range=(1,1),
                             max_features=15000,
                            token_pattern =r'\w{1,}')
vectorizer_word.fit(text)
X_train_word = vectorizer_word.transform(train['comment_text'])
X_test_word = vectorizer_word.transform(test['comment_text'])

# char vectorize 
vectorizer_char = TfidfVectorizer(sublinear_tf=True, 
                                  strip_accents='unicode',
                                  analyzer='char',
                                  ngram_range=(1,5), 
                                  max_features=20000)
vectorizer_char.fit(text)
X_train_char = vectorizer_char.transform(train['comment_text'])
X_test_char = vectorizer_char.transform(test['comment_text'])

X_tr=hstack([X_train_word,X_train_char])
X_te = hstack([X_test_word,X_test_char])

NameError: name 'hstack' is not defined

In [16]:
X_tr=hstack([X_train_word,X_train_char])
X_te = hstack([X_test_word,X_test_char])

In [19]:
# logistic regression model development
classifier = LogisticRegression(solver='sag')
submission = test[['id']]
for class_name in labels:
    
    y= train[class_name]
    #cross validation
    cv_score = np.mean(cross_val_score(classifier, X_tr, y, cv=3, scoring='roc_auc'))
    print('CV score for class {} is {}'.format(class_name,cv_score))
    
    classifier.fit(X_tr,y)
    submission[class_name] = classifier.predict_proba(X_te)[:,1]

CV score for class toxic is 0.9781568348341478
CV score for class severe_toxic is 0.9885459662650101
CV score for class obscene is 0.9899492539319966
CV score for class threat is 0.988919315768867
CV score for class insult is 0.9824691986285377
CV score for class identity_hate is 0.9827104316348306


In [30]:
submission.to_csv('data/submission.csv',index=False)